In [ ]:
# COSMIC Data - Dataset processing (into multidimensional bins)
# Dataset from Chinese group (downloaded from COSMIC web page)

# Processing whole dataset as one, and separately year by year to get standard deviations
# Both normal lat-lon coordinates as well as magnetic lat-lon coords

# Can turn on fivedegbins switch to use 5x5 deg lat lon bins

In [1]:
import xarray as xr
import numpy as np
import cftime
import nc_time_axis
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import ticker, cm
#from cftime import datetime 
from datetime import datetime, timedelta
#import datetime as dt
import matplotlib.colors as colors
import math
import random
import matplotlib.cm as mcm
from matplotlib.ticker import MaxNLocator
#jet = mcm.get_cmap('jet')
jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')
import netCDF4 as nc
import sys
import os
import psutil
import netCDF4 as nc
import cartopy.crs as ccrs # CRS stands for "Coordinate reference systems" for map projection
from cartopy.crs import PlateCarree
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from dateutil import tz
import pytz
import time
from time import process_time
from tqdm import tqdm
from dask.distributed import LocalCluster, Client
import dask.array as da
import dask.dataframe as dd
import dask
from dask import delayed
from dask.diagnostics import ProgressBar  # Import the ProgressBar class
import calendar
%matplotlib inline 
#import line_profiler
#%load_ext line_profiler

/tmp/ipykernel_2375281/2706121075.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')


# Open WACCM dataset (& 5' bins switch)

In [2]:
fivedegbins = 1

################################################################################
# Open WACCM dataset to define bins
################################################################################

# arrays sliced between chosen range (~90-130km) indices 42-60, 19 levels

file1name='Nc_Files/SpE_Output/Wuhu_IonTr_run_SpE_Output_Dec-Feb.nc' 
wds = xr.open_dataset(file1name)

# wds_ilev = wds['ilev']
# wds_time = wds['time']

wds_lat = wds['lat']
wds_lon = wds['lon']

lev_sl = wds['lev_sl']
Zavg_sl = wds.variables['Zavg_sl'] 
 
altavg_sl = wds['altavg_sl']
# altavg_sl.values # array([131.1492157 , 126.73989868, 122.75083923, 119.16303253,
#        115.95503998, 113.10031128, 110.56510162, 108.3082428 ,
#        106.28289795, 104.44081879, 102.7361145 , 101.12830353,
#         99.58457184,  98.0800705 ,  96.59645081,  95.12115479,
#         93.64645386,  92.16801453,  90.6837616 ])


print('    Array lev = ' + str("%.1e" % lev_sl[-1] ) + 'hPa : ' + str("%.1e" % lev_sl[0] ) + 'hPa'
           + ' (Z3 approx ' + str("%.0f" % Zavg_sl[-1]) + '-' + str("%.0f" % Zavg_sl[0] ) + 'km' + ',' 
           + ' Alt approx ' + str("%.0f" % altavg_sl[-1]) + '-' + str("%.0f" % altavg_sl[0] ) + 'km' + ')' )


    Array lev = 1.3e-03hPa : 1.5e-05hPa (Z3 approx 89-129km, Alt approx 91-131km)


In [3]:
# OLD - Replaced by the cell above. Leaving for now in case any needed below

# fivedegbins = 1

# ################################################################################
# # Open WACCM dataset to define bins
# ################################################################################


# # Define the relative path to the symbolic link
# symbolic_link = 'Nc_Files/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport'

# # Get the full path to the symbolic link
# link_path = os.path.expanduser(os.path.expandvars(os.path.normpath(symbolic_link)))

# # Join the symbolic link with the specific file you want to open
# file_name = 'ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport.cam.h2.0001-01-01-00000.nc'
# file_path = os.path.join(link_path, file_name)

# # Open the file
# wds = xr.open_dataset(file_path)
# #wds

# wds_lat = wds['lat']
# wds_lon = wds['lon']
# wds_lev = wds['lev']
# wds_ilev = wds['ilev']
# wds_time = wds['time']

# wds_geopH = wds.variables['Z3'] / 1000 #m-> km
# wds_Zavg = wds_geopH.mean(('time','lat', 'lon'))

# Re = 6378 #km
# wds_alt = (wds_geopH*Re)/(Re-wds_geopH)
# wds_altavg = wds_alt.mean(('time','lat', 'lon')) 

# # Slice arrays (lev, altitude) between chosen range (~80-130km)
# # lev_sl_idx_min = 42
# # lev_sl_idx_max = 66       #25 levels

# lev_sl_idx_min = 42
# lev_sl_idx_max = 60         #19 levels

# # Slice arrays (lev & alt) between chosen range & print
# lev_sl = wds_lev[lev_sl_idx_min:lev_sl_idx_max+1]
# Zavg_sl = wds_Zavg[lev_sl_idx_min:lev_sl_idx_max+1]
# altavg_sl = wds_altavg[lev_sl_idx_min:lev_sl_idx_max+1]

# print('    Array lev = ' + str("%.1e" % wds_lev[lev_sl_idx_max] ) + 'hPa : ' + str("%.1e" % wds_lev[lev_sl_idx_min] ) + 'hPa'
#            + ' (Z3 approx ' + str("%.0f" % wds_Zavg[lev_sl_idx_max]) + 'km : ' + str("%.0f" % wds_Zavg[lev_sl_idx_min] ) + 'km' + ')' 
#            + ' (Alt approx ' + str("%.0f" % wds_altavg[lev_sl_idx_max]) + 'km : ' + str("%.0f" % wds_altavg[lev_sl_idx_min] ) + 'km' + ')' )


# Defining Lat Bins

In [4]:
################################################################################
# Lat bins
################################################################################

lat_values = wds_lat.values

lat_bin_edges = lat_values
lat_bin_midpoints = (lat_bin_edges[:-1] + lat_bin_edges[1:]) / 2
#lat_bin_widths = np.diff(lat_bin_edges)
#lat_average_bin_width = np.mean(lat_bin_widths)
lat_num_bins = len(lat_bin_midpoints)


# # ##Print the results
# print("Lat Bin Edges (Lower and Upper):")
# for i in range(len(lat_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{lat_bin_edges[i]}, {lat_bin_edges[i + 1]}]")

# print("\nLat Bin Midpoints:")
# for i in range(len(lat_bin_midpoints)):
#     print(f"Bin {i + 1} Midpoint: {lat_bin_midpoints[i]}")

# print("\nLat Bin Widths:", lat_bin_widths)
# print(f"Lat Average Bin Width: {lat_average_bin_width:.6f} degrees")
# print(f"Lat Number of Bins: {lat_num_bins}")


# # Lat Average Bin Width: 1.894737 degrees
# # Lat Number of Bins: 95

# Defining Lon Bins

In [5]:
# ################################################################################
# # Lon bins -180 -> 180
# ################################################################################

# Updated value for a range of -180 to +180 degrees
lon_upper_edge_last_bin = 180.0
lon_bin_edges = np.append(np.arange(-180.0, 180.0, 2.5), lon_upper_edge_last_bin)
lon_bin_midpoints = (lon_bin_edges[:-1] + lon_bin_edges[1:]) / 2

# Other calculations (width, number of bins, etc.) remain the same
lon_bin_widths = np.diff(lon_bin_edges)
lon_average_bin_width = np.mean(lon_bin_widths)
lon_num_bins = len(lon_bin_midpoints)

# # Print the results
# print("Lon Bin Edges (Lower and Upper):")
# for i in range(len(lon_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{lon_bin_edges[i]}, {lon_bin_edges[i + 1]}]")

# print("\nLon Bin Midpoints:")
# for i in range(len(lon_bin_midpoints)):
#     print(f"Lon Bin {i + 1} Midpoint: {lon_bin_midpoints[i]}")

# print("\nLon Bin Widths:", lon_bin_widths)
# print(f"Average Lon Bin Width: {lon_average_bin_width:.8f} degrees")
# print(f"Number of Lon Bins: {lon_num_bins}")

# Defining 5 Degree Bins 

In [6]:
################################################################################
# 5 deg Lat bins
################################################################################

if fivedegbins == 1:
    lat_values = np.arange(-90, 95, 5)
    lat_bin_edges = lat_values
    lat_bin_midpoints = (lat_bin_edges[:-1] + lat_bin_edges[1:]) / 2
    lat_num_bins = len(lat_bin_midpoints)

    lon_bin_edges = np.arange(-180, 185, 5)
    lon_bin_midpoints = (lon_bin_edges[:-1] + lon_bin_edges[1:]) / 2
    lon_num_bins = len(lon_bin_midpoints) #72

# Defining Altitude Bins

In [7]:
################################################################################
# Zavg_sl bins - Note first bin is at top
################################################################################

# Convert xarray Variable to NumPy array
Zavg_sl_values = Zavg_sl.values

Zavg_sl_bin_widths = np.diff(Zavg_sl_values)
Zavg_sl_bin_edges = [Zavg_sl_values[0] - Zavg_sl_bin_widths[0] / 2] + [Zavg_sl_values[i] + Zavg_sl_bin_widths[i] / 2 for i in range(len(Zavg_sl_bin_widths))] + [Zavg_sl_values[-1] + Zavg_sl_bin_widths[-1] / 2]
Zavg_sl_bin_midpoints = [(Zavg_sl_bin_edges[i] + Zavg_sl_bin_edges[i + 1]) / 2 for i in range(len(Zavg_sl_bin_edges) - 1)]
#Zavg_sl_bin_widths = [Zavg_sl_bin_edges[i + 1] - Zavg_sl_bin_edges[i] for i in range(len(Zavg_sl_bin_edges) - 1)]
Zavg_sl_num_bins = len(Zavg_sl_bin_midpoints)


# # Print the bin edges and midpoints
# print("Zavg_sl Bin Edges (Lower and Upper):")
# for i in range(len(Zavg_sl_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{Zavg_sl_bin_edges[i]}, {Zavg_sl_bin_edges[i + 1]}")

# print("\nZavg_sl Bin Midpoints:")
# for i in range(len(Zavg_sl_bin_midpoints)):
#     print(f"Bin {i + 1} Midpoint: {Zavg_sl_bin_midpoints[i]}")

# # Print the bin widths
# print("\nZavg_sl Bin Widths:")
# for i in range(len(Zavg_sl_bin_widths)):
#     print(f"Bin {i + 1} Width: {Zavg_sl_bin_widths[i]}")

# print(f"Number of Bins: {Zavg_sl_num_bins}")

# ### Number of Zavg_sl_bin_widths Bins: 20 (25 previously)

# Defining LT Bins

In [8]:
LT_bin_edges = np.linspace(0, 24, 49)
LT_bin_edges[-1] = 24.0

# Calculate bin midpoints
LT_bin_midpoints = (LT_bin_edges[:-1] + LT_bin_edges[1:]) / 2

# Calculate the number of bins
LT_num_bins = len(LT_bin_edges) - 1

# Calculate bin widths
LT_bin_widths = np.diff(LT_bin_edges)

# Calculate the average bin width
average_bin_width = np.sum(LT_bin_widths) / LT_num_bins


# ##Print the results
# print("LT Bin Edges (Lower and Upper):")
# for i in range(len(LT_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{LT_bin_edges[i]}, {LT_bin_edges[i + 1]}]")

# print("\nLT Bin Midpoints:")
# for i in range(len(LT_bin_midpoints)):
#     print(f"Bin {i + 1} Midpoint: {LT_bin_midpoints[i]}")

# print("\nNumber of LT Bins:")
# print(LT_num_bins)

# # Print the average bin width
# print("Average LT Bin Width:", average_bin_width)

# Raw satellite data - Opening ds

In [9]:
### Raw satellite data ###
filepath = 'Nc_Files/s4max_files/mod_s4max.nc'

### Processed time data into datetime format ###
#new_filepath = 'Nc_Files/s4max_files/mod_s4max.nc'

### My processed data - 80-130km only ###
#result_filepath = 'Nc_Files/s4max_files/mod_s4max_80-130km.nc'

ds = xr.open_dataset(filepath)
#ds

#  NORMAL  Lat Lon coordinates - Processing

In [10]:
####################################      N O R M A L  Lat Lon coordinates     ##############################################

# Define evalue_threshold for each season
evalue_thresholds = {
    'winter': 1.50998e+03,
    'spring': 1.50998e+03,
    'summer': 1.50998e+03,
    'autumn': 2.39315e+03
}

s4max_threshold = 0.35

#Arras and Wickert 2018 s4max > 0.2
#Yue et al 2016 s4max > 0.3
#Yue et al 2015 s4max > 0.3
#Ray 2006 s4max > 0.4
#Ko and Yeh 2010 'strong scintillations' defined as smax >= 0.5

###########################################################################################################################
# Define multiple sets of desired months
all_desired_months = [ [3, 4, 5], [6, 7, 8], [9, 10, 11] , [12, 1, 2] ]

# Loop over each set of desired months
for desired_months in all_desired_months:
    print('Months ' + str(desired_months))
    averaging_period = ""
    if len(desired_months) == 1:
        averaging_period = "Monthly"
    elif len(desired_months) == 2:
        averaging_period = "Two-Month"
    elif len(desired_months) == 3:
        averaging_period = "Three-Month"
    else:
        averaging_period = "Unknown"

    season = ""
    if set(desired_months) <= {12, 1, 2}:
        season = "winter"
    elif set(desired_months) <= {3, 4, 5}:
        season = "spring"
    elif set(desired_months) <= {6, 7, 8}:
        season = "summer"
    elif set(desired_months) <= {9, 10, 11}:
        season = "autumn"
    else:
        season = "unknown"
        
    # Access the appropriate evalue_threshold for the current season
    current_evalue_threshold = evalue_thresholds[season]
    
    freq_LT = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))
    nmeas_LT = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))

    freq_lon = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))
    nmeas_lon = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))

    nfile = len(ds['lat'])
    ds_length = np.arange(nfile)

    ###########################################################################################################################
    for i in tqdm(ds_length):
        current_altitude = ds['altitude_var'].values[i]
        current_lat = ds['lat'].values[i]
        current_lon = ds['lon'].values[i]
        current_LT = ds["LT"].values[i]
        current_s4max = ds['s4max'].values[i]

        current_time = ds["time"].values[i]
        current_month = int(np.datetime64(current_time, 'M').astype('M8[M]').view('i8')) % 12 + 1
        
        evalue = ( ( ( np.sqrt(13.64*current_s4max) + 1.2 ) / 8.98 )**2 ) * 1e6
                  
        if current_month in desired_months:
            if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[0]:
                altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges, right=False) - 1
                lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
                lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
                if current_lon == lon_bin_edges[-1]:
                    lon_bin_index = len(lon_bin_edges) - 2
                LT_bin_index = np.digitize(current_LT, LT_bin_edges) - 1
                if current_LT == LT_bin_edges[-1]:
                    LT_bin_index = len(LT_bin_edges) - 2

                nmeas_lon[altitude_bin_index, lat_bin_index, lon_bin_index] += 1.
                nmeas_LT[altitude_bin_index, lat_bin_index, LT_bin_index] += 1.

                if current_s4max > s4max_threshold and evalue > current_evalue_threshold:
                    freq_lon[altitude_bin_index, lat_bin_index, lon_bin_index] += 1.
                    freq_LT[altitude_bin_index, lat_bin_index, LT_bin_index] += 1.

    ###########################################################################################################################
    # Calculate occurence frequency
    ocfr_lon = np.full((Zavg_sl_num_bins, lat_num_bins, lon_num_bins), 0.)
    ocfr_LT = np.full((Zavg_sl_num_bins, lat_num_bins, LT_num_bins), 0.)

    # Check if arrays are equal to 0
    lon_mask = (freq_lon != 0.) & (nmeas_lon != 0.)
    LT_mask = (freq_LT != 0.) & (nmeas_LT != 0.)

    # Calculate ocfr_lon and ocfr_LT only where the mask is True
    if lon_mask.any():
        ocfr_lon[lon_mask] = (freq_lon[lon_mask] / nmeas_lon[lon_mask]) * 100.0

    if LT_mask.any():
        ocfr_LT[LT_mask] = (freq_LT[LT_mask] / nmeas_LT[LT_mask]) * 100.0

        
    std_dev_ocfr_lon = ocfr_lon.std(axis=0)
    std_dev_ocfr_LT = ocfr_LT.std(axis=0)

    
    ###########################################################################################################################
    # Save occurrence frequency to nc file
    Ocfr_ds = xr.Dataset(
        data_vars={
            "ocfr_lon": (['altitude', 'latitude', 'longitude'], ocfr_lon),
            "ocfr_LT": (['altitude', 'latitude', 'local_time'], ocfr_LT)
        },
        coords={
            "altitude": Zavg_sl_bin_midpoints,
            "latitude": lat_bin_midpoints,
            "longitude": lon_bin_midpoints,
            "local_time": LT_bin_midpoints,
        },
        attrs={
            'averaging_period': averaging_period,
            'season': season
        }
    )

    output_directory = "./Nc_Files/s4max/e_criteria/"
    output_file = f"{output_directory}Ocfr_{season}_s4max_{s4max_threshold}.nc"
    if fivedegbins == 1:
        output_file = f"{output_directory}Ocfr_{season}_5d_s4max_{s4max_threshold}.nc"
    Ocfr_ds.to_netcdf(output_file)

###########################################################################################################################


Months [3, 4, 5]


100%|████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [12:33<00:00, 12291.56it/s]


Months [6, 7, 8]


100%|████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [12:39<00:00, 12199.58it/s]


Months [9, 10, 11]


100%|████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [12:33<00:00, 12299.26it/s]


Months [12, 1, 2]


100%|████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [12:35<00:00, 12258.14it/s]


# MAGNETIC Lat Lon coordinates - Processing

In [11]:
################################       M A G N E T I C Lat Lon coordinates        ##########################################

# Define evalue_threshold for each season
evalue_thresholds = {
    'winter': 1.50998e+03,
    'spring': 1.50998e+03,
    'summer': 1.50998e+03,
    'autumn': 2.39315e+03
}

s4max_threshold = 0.35

#Arras and Wickert 2018 s4max > 0.2
#Yue et al 2016 s4max > 0.3
#Yue et al 2015 s4max > 0.3
#Ray 2006 s4max > 0.4
#Ko and Yeh 2010 'strong scintillations' defined as smax >= 0.5

###########################################################################################################################
# Define multiple sets of desired months
all_desired_months = [ [3, 4, 5], [6, 7, 8], [9, 10, 11] , [12, 1, 2] ]

# Loop over each set of desired months
for desired_months in all_desired_months:
    print('Months ' + str(desired_months))
    averaging_period = ""
    if len(desired_months) == 1:
        averaging_period = "Monthly"
    elif len(desired_months) == 2:
        averaging_period = "Two-Month"
    elif len(desired_months) == 3:
        averaging_period = "Three-Month"
    else:
        averaging_period = "Unknown"

    season = ""
    if set(desired_months) <= {12, 1, 2}:
        season = "winter"
    elif set(desired_months) <= {3, 4, 5}:
        season = "spring"
    elif set(desired_months) <= {6, 7, 8}:
        season = "summer"
    elif set(desired_months) <= {9, 10, 11}:
        season = "autumn"
    else:
        season = "unknown"
        
    # Access the appropriate evalue_threshold for the current season
    current_evalue_threshold = evalue_thresholds[season]
 
    freq_LT_m = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))
    nmeas_LT_m = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))

    freq_lon_m = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))
    nmeas_lon_m = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))

    nfile = len(ds['lat'])
    ds_length = np.arange(nfile)

    ###########################################################################################################################
    for i in tqdm(ds_length):
        current_altitude = ds['altitude_var'].values[i]
        # current_lat = ds['lat'].values[i]
        # current_lon = ds['lon'].values[i]
        current_LT = ds["LT"].values[i]
        current_s4max = ds['s4max'].values[i]
        current_mlat = ds['mlat'].values[i]
        current_mlon = ds['mlon'].values[i]

        current_time = ds["time"].values[i]
        current_month = int(np.datetime64(current_time, 'M').astype('M8[M]').view('i8')) % 12 + 1

        evalue = ( ( ( np.sqrt(13.64*current_s4max) + 1.2 ) / 8.98 )**2 ) * 1e6
        
        if current_month in desired_months:
            if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[0]:
                altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges, right=False) - 1
                mlat_bin_index = np.digitize(current_mlat, lat_bin_edges) - 1
                mlon_bin_index = np.digitize(current_mlon, lon_bin_edges) - 1
                
                if current_mlon == lon_bin_edges[-1]:
                    mlon_bin_index = len(lon_bin_edges) - 2
                    
                LT_bin_index = np.digitize(current_LT, LT_bin_edges) - 1
                if current_LT == LT_bin_edges[-1]:
                    LT_bin_index = len(LT_bin_edges) - 2

                nmeas_lon_m[altitude_bin_index, mlat_bin_index, mlon_bin_index] += 1.
                nmeas_LT_m[altitude_bin_index, mlat_bin_index, LT_bin_index] += 1.

                if current_s4max > s4max_threshold and evalue > current_evalue_threshold:
                    freq_lon_m[altitude_bin_index, mlat_bin_index, mlon_bin_index] += 1.
                    freq_LT_m[altitude_bin_index, mlat_bin_index, LT_bin_index] += 1.

    ###########################################################################################################################
    # Calculate occurence frequency
    ocfr_lon_m = np.full((Zavg_sl_num_bins, lat_num_bins, lon_num_bins), 0.)
    ocfr_LT_m = np.full((Zavg_sl_num_bins, lat_num_bins, LT_num_bins), 0.)

    # Check if arrays are equal to 0
    lon_mask_m = (freq_lon_m != 0.) & (nmeas_lon_m != 0.)
    LT_mask_m = (freq_LT_m != 0.) & (nmeas_LT_m != 0.)

    # Calculate ocfr_lon and ocfr_LT only where the mask is True
    if lon_mask_m.any():
        ocfr_lon_m[lon_mask_m] = (freq_lon_m[lon_mask_m] / nmeas_lon_m[lon_mask_m]) * 100.0

    if LT_mask_m.any():
        ocfr_LT_m[LT_mask_m] = (freq_LT_m[LT_mask_m] / nmeas_LT_m[LT_mask_m]) * 100.0

        
     #*****************   
        
    ###########################################################################################################################
    # Save occurrence frequency to nc file
    Ocfr_ds = xr.Dataset(
        data_vars={
            "ocfr_lon_m": (['altitude', 'magnetic latitude', 'magnetic longitude'], ocfr_lon_m),
            "ocfr_LT_m": (['altitude', 'magnetic latitude', 'local_time'], ocfr_LT_m)
        },
        coords={
            "altitude": Zavg_sl_bin_midpoints,
            "magnetic latitude": lat_bin_midpoints,
            "magnetic longitude": lon_bin_midpoints,
            "local_time": LT_bin_midpoints,
        },
        attrs={
            'averaging_period': averaging_period,
            'season': season
        }
    )

    output_directory = "./Nc_Files/s4max/e_criteria/maglat_maglon/"
    output_file = f"{output_directory}Ocfr_m_{season}_s4max_{s4max_threshold}.nc"
    if fivedegbins == 1:
        output_file = f"{output_directory}Ocfr_m_{season}_5d_s4max_{s4max_threshold}.nc"
    Ocfr_ds.to_netcdf(output_file)

###########################################################################################################################


Months [3, 4, 5]


100%|████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [12:21<00:00, 12495.48it/s]


Months [6, 7, 8]


100%|████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [12:23<00:00, 12457.29it/s]


Months [9, 10, 11]


100%|████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [12:19<00:00, 12532.45it/s]


Months [12, 1, 2]


100%|████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [12:26<00:00, 12415.53it/s]


# MULTI-YEARS - NORMAL Lat Lon - Processing (all seasons)

In [ ]:
####################################      N O R M A L  Lat Lon coordinates     ##############################################
####################################     M U L T I - Y E A R S  OF  D A T A    ##############################################
####################################           PROCESSED SEPARATELY            ##############################################

# Define evalue_threshold for each season
evalue_thresholds = {
    'winter': 1.50998e+03,
    'spring': 1.50998e+03,
    'summer': 1.50998e+03,
    'autumn': 2.39315e+03
}

s4max_threshold = 0.35

#Arras and Wickert 2018 s4max > 0.2
#Yue et al 2016 s4max > 0.3
#Yue et al 2015 s4max > 0.3
#Ray 2006 s4max > 0.4
#Ko and Yeh 2010 'strong scintillations' defined as smax >= 0.5

###########################################################################################################################

year_desired_months = [ [3, 4, 5] , [6, 7, 8], [9, 10, 11], [12, 1,2] ]
all_desired_months = []

for d, seasonmonths in enumerate(year_desired_months):

    if set(seasonmonths) <= {3, 4, 5}:
        all_desired_months = [ [3, 4, 5] ]
        years_set = [ '2007' , '2008', '2009', '2010', '2011', '2012', '2013' ]
        MAM_date_sets = [
            ['2007-03-01', '2007-05-31'],
            ['2008-03-01', '2008-05-31'],
            ['2009-03-01', '2009-05-31'],
            ['2010-03-01', '2010-05-31'],
            ['2011-03-01', '2011-05-31'],
            ['2012-03-01', '2012-05-31'],
            ['2013-03-01', '2013-05-31']  
            ]      # x 7 years
        target_date_sets = [[pd.to_datetime(date) for date in set_dates] for set_dates in MAM_date_sets]

    elif set(seasonmonths) <= {6, 7, 8}:
        all_desired_months = [ [6, 7, 8] ]
        years_set = [ '2007', '2008', '2009', '2010', '2011', '2012', '2013'  ]
        JJA_date_sets = [
            ['2007-06-01', '2007-08-31'],
            ['2008-06-01', '2008-08-31'],
            ['2009-06-01', '2009-08-31'],
            ['2010-06-01', '2010-08-31'],
            ['2011-06-01', '2011-08-31'],
            ['2012-06-01', '2012-08-31'],
            ['2013-06-01', '2013-08-31']  
            ]      # x 7 years
        target_date_sets = [[pd.to_datetime(date) for date in set_dates] for set_dates in JJA_date_sets]

    elif set(seasonmonths) <= {9, 10, 11}:
        all_desired_months = [ [9, 10, 11] ]
        years_set = [ '2007', '2008', '2009', '2010', '2011', '2012', '2013' ]
        SON_date_sets = [
            ['2007-09-01', '2007-11-30'],
            ['2008-09-01', '2008-11-30'],
            ['2009-09-01', '2009-11-30'],
            ['2010-09-01', '2010-11-30'],
            ['2011-09-01', '2011-11-30'],
            ['2012-09-01', '2012-11-30'],
            ['2013-09-01', '2013-11-30']  ]      # x 7 years
        target_date_sets = [[pd.to_datetime(date) for date in set_dates] for set_dates in SON_date_sets]

    elif set(seasonmonths) <= {12, 1, 2}:
        all_desired_months = [ [12, 1, 2] ]
        years_set = [ '2007-8', '2008-9', '2009-10', '2010-11', '2011-12', '2012-13' ]
        DJF_date_sets = [
            ['2007-12-01', '2008-02-29'],
            ['2008-12-01', '2009-02-28'],
            ['2009-12-01', '2010-02-28'],
            ['2010-12-01', '2011-02-28'],
            ['2011-12-01', '2012-02-29'],
            ['2012-12-01', '2013-02-28']  ]      # x 6 years
        target_date_sets = [[pd.to_datetime(date) for date in set_dates] for set_dates in DJF_date_sets]
    else:
        print('error error error')

    # Convert 'time' coordinate values to datetime objects
    time_values = pd.to_datetime(ds['time'].values)

    # Find the index of the first value for the first date and the last index for the second date in each set
    indices_for_target_dates_sets = [
        [np.where(time_values >= set_dates[0])[0].min(), np.where(time_values <= set_dates[1])[0].max()] for set_dates in target_date_sets
    ]

    # Display the indices for each set of target dates
    for i, indices_for_target_dates in enumerate(indices_for_target_dates_sets):
        print(f'{years_set[i]} {all_desired_months}:')
        print(f"Indices for Target Dates Set {i + 1}: {indices_for_target_dates}")



    for i, indices_for_target_dates in enumerate(indices_for_target_dates_sets):

        dssl_alt = ds['altitude_var'][indices_for_target_dates[0]: indices_for_target_dates[1] + 1]
        dssl_lat = ds['lat'][indices_for_target_dates[0]: indices_for_target_dates[1] + 1]
        dssl_lon = ds['lon'][indices_for_target_dates[0]: indices_for_target_dates[1] + 1]
        dssl_LT = ds["LT"][indices_for_target_dates[0]: indices_for_target_dates[1] + 1]
        dssl_s4max = ds['s4max'][indices_for_target_dates[0]: indices_for_target_dates[1] + 1]
        dssl_time = ds["time"][indices_for_target_dates[0]: indices_for_target_dates[1] + 1]

        year = years_set[i]
        print('Year ' + str(year))

        # Loop over each set of desired months
        for desired_months in all_desired_months:
            print('Months ' + str(desired_months))
            averaging_period = ""
            if len(desired_months) == 1:
                averaging_period = "Monthly"
            elif len(desired_months) == 2:
                averaging_period = "Two-Month"
            elif len(desired_months) == 3:
                averaging_period = "Three-Month"
            else:
                averaging_period = "Unknown"

            season = ""
            if set(desired_months) <= {12, 1, 2}:
                season = "winter"
            elif set(desired_months) <= {3, 4, 5}:
                season = "spring"
            elif set(desired_months) <= {6, 7, 8}:
                season = "summer"
            elif set(desired_months) <= {9, 10, 11}:
                season = "autumn"
            else:
                season = "unknown"

            # Access the appropriate evalue_threshold for the current season
            current_evalue_threshold = evalue_thresholds[season]
    
            freq_LT = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))
            nmeas_LT = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))

            freq_lon = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))
            nmeas_lon = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))

            nfile = len(dssl_lat)
            ds_length = np.arange(nfile)

            ###########################################################################################################################
            for i in tqdm(ds_length):
                current_altitude = dssl_alt[i]
                current_lat = dssl_lat[i]
                current_lon = dssl_lon[i]
                current_LT = dssl_LT[i]
                current_s4max = dssl_s4max[i]
                current_time = dssl_time[i]

                #current_month = int(np.datetime64(current_time, 'M').astype('M8[M]').view('i8')) % 12 + 1
                #current_month = int(current_time.astype('M8[M]').view('i8')) % 12 + 1
                current_month = int(current_time.dt.month)  # Extract month as an integer

                evalue = ( ( ( np.sqrt(13.64*current_s4max) + 1.2 ) / 8.98 )**2 ) * 1e6

                if current_month in desired_months:
                    if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[0]:
                        altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges, right=False) - 1
                        lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
                        lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
                        if current_lon == lon_bin_edges[-1]:
                            lon_bin_index = len(lon_bin_edges) - 2
                        LT_bin_index = np.digitize(current_LT, LT_bin_edges) - 1
                        if current_LT == LT_bin_edges[-1]:
                            LT_bin_index = len(LT_bin_edges) - 2

                        nmeas_lon[altitude_bin_index, lat_bin_index, lon_bin_index] += 1.
                        nmeas_LT[altitude_bin_index, lat_bin_index, LT_bin_index] += 1.

                        if current_s4max > s4max_threshold and evalue > current_evalue_threshold:
                            freq_lon[altitude_bin_index, lat_bin_index, lon_bin_index] += 1.
                            freq_LT[altitude_bin_index, lat_bin_index, LT_bin_index] += 1.

            ###########################################################################################################################
            # Calculate occurence frequency
            ocfr_lon = np.full((Zavg_sl_num_bins, lat_num_bins, lon_num_bins), 0.)
            ocfr_LT = np.full((Zavg_sl_num_bins, lat_num_bins, LT_num_bins), 0.)

            # Check if arrays are equal to 0
            lon_mask = (freq_lon != 0.) & (nmeas_lon != 0.)
            LT_mask = (freq_LT != 0.) & (nmeas_LT != 0.)

            # Calculate ocfr_lon and ocfr_LT only where the mask is True
            if lon_mask.any():
                ocfr_lon[lon_mask] = (freq_lon[lon_mask] / nmeas_lon[lon_mask]) * 100.0

            if LT_mask.any():
                ocfr_LT[LT_mask] = (freq_LT[LT_mask] / nmeas_LT[LT_mask]) * 100.0

            ###########################################################################################################################
            # Save occurrence frequency to nc file
            Ocfr_ds = xr.Dataset(
                data_vars={
                    "ocfr_lon": (['altitude', 'latitude', 'longitude'], ocfr_lon),
                    "ocfr_LT": (['altitude', 'latitude', 'local_time'], ocfr_LT)
                },
                coords={
                    "altitude": Zavg_sl_bin_midpoints,
                    "latitude": lat_bin_midpoints,
                    "longitude": lon_bin_midpoints,
                    "local_time": LT_bin_midpoints,
                },
                attrs={
                    'year': year,
                    'averaging_period': averaging_period,
                    'season': season
                }
            )

            output_directory = "./Nc_Files/s4max/e_criteria/individual_yrs/"
            output_file = f"{output_directory}Ocfr_{year}_{season}_s4max_{s4max_threshold}.nc"
            if fivedegbins == 1:
                output_file = f"{output_directory}Ocfr_{year}_{season}_5d_s4max_{s4max_threshold}.nc"
            Ocfr_ds.to_netcdf(output_file)

        ###########################################################################################################################


2007 [[3, 4, 5]]:
Indices for Target Dates Set 1: [256970, 739018]
2008 [[3, 4, 5]]:
Indices for Target Dates Set 2: [1652474, 2017396]
2009 [[3, 4, 5]]:
Indices for Target Dates Set 3: [3309162, 3742893]
2010 [[3, 4, 5]]:
Indices for Target Dates Set 4: [4945053, 5300255]
2011 [[3, 4, 5]]:
Indices for Target Dates Set 5: [6115790, 6415241]
2012 [[3, 4, 5]]:
Indices for Target Dates Set 6: [7203654, 7443213]
2013 [[3, 4, 5]]:
Indices for Target Dates Set 7: [8236216, 8522674]
Year 2007
Months [3, 4, 5]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 482049/482049 [1:13:55<00:00, 108.69it/s]


Year 2008
Months [3, 4, 5]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 364923/364923 [47:26<00:00, 128.22it/s]


Year 2009
Months [3, 4, 5]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380860/380860 [50:59<00:00, 124.50it/s]


Year 2008
Months [6, 7, 8]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 434071/434071 [1:02:00<00:00, 116.68it/s]


Year 2009
Months [6, 7, 8]


 45%|███████████████████████████████████████████████████████████████████████▎                                                                                        | 178357/400139 [24:11<30:20, 121.85it/s]

# MULTI YEARS - NORMAL Lat Lon - Standard Deviation over time 

In [20]:
# Standard Deviation over time (multiple years)

s4max_threshold = 0.35

# Define seasons, years, and output directory
seasons = [
    #'spring', 'summer'], 
    'autumn'#, 
#'winter'
]
years = list(range(2007, 2014))  # For spring, summer, and autumn
years_winter = list(range(2007, 2013))  # For winter
file_directory = "./Nc_Files/s4max/e_criteria/individual_yrs/"

# Initialize an empty dataset to store standard deviations
std_dev_dataset = xr.Dataset()

# Loop through seasons
for season in seasons:
    # Define a list to store datasets for each year
    datasets = []

    # Loop through years
    if season == 'winter':
        years_to_loop = years_winter
    else:
        years_to_loop = years

    for year in years_to_loop:
        # Define the file pattern
        if season == 'winter':
            yr_last_two_digits = str(int(str(year)[-2:]) + 1)
            file_pattern = f"{file_directory}Ocfr_{year}-{yr_last_two_digits}_{season}_5d_s4max_{s4max_threshold}.nc"
        else:
            file_pattern = f"{file_directory}Ocfr_{year}_{season}_5d_s4max_{s4max_threshold}.nc"

        # Open the dataset
        ocfr_ds = xr.open_dataset(file_pattern)

        # Add a 'time' coordinate representing the year
        ocfr_ds['time'] = xr.DataArray([year], dims='time', coords={'time': [year]})

        # Append the dataset to the list
        datasets.append(ocfr_ds)

    # Concatenate along the time dimension
    combined_ds = xr.concat(datasets, dim='time')

    # Calculate the standard deviation for ocfr_lon and ocfr_LT
    std_dev_lon = combined_ds['ocfr_lon'].std(dim='time')
    std_dev_LT = combined_ds['ocfr_LT'].std(dim='time')

    # Add the standard deviations to the result dataset
    std_dev_dataset[f'std_dev_ocfr_lon_{season}'] = std_dev_lon
    std_dev_dataset[f'std_dev_ocfr_LT_{season}'] = std_dev_LT

# Print or save the result dataset
#print(std_dev_dataset)

output_directory = "./Nc_Files/s4max/e_criteria/individual_yrs/"
output_file = f"{output_directory}Ocfr_stddev_{season}_s4max_{s4max_threshold}.nc"


try:
    std_dev_dataset.to_netcdf(output_file)
    print(f"File saved successfully: {output_file}")
except Exception as e:
    print(f"Error saving the file: {e}")
finally:
    std_dev_dataset.close()

FileNotFoundError: [Errno 2] No such file or directory: b'/localhome/sestay/jupyter_new/Nc_Files/s4max/COSMIC_individual_yrs/Ocfr_2007_autumn_5d_s4max_0.35.nc'

In [19]:
stdev_ds = xr.open_dataset(output_file)
stdev_ds
#stdev_ds['std_dev_ocfr_lon_spring'].values

<xarray.Dataset>
Dimensions:                  (altitude: 25, latitude: 36, longitude: 72,
                              local_time: 48)
Coordinates:
  * altitude                 (altitude) float64 126.2 122.5 ... 82.44 81.07
  * latitude                 (latitude) float64 -87.5 -82.5 -77.5 ... 82.5 87.5
  * longitude                (longitude) float64 -177.5 -172.5 ... 172.5 177.5
  * local_time               (local_time) float64 0.25 0.75 1.25 ... 23.25 23.75
Data variables:
    std_dev_ocfr_lon_spring  (altitude, latitude, longitude) float64 ...
    std_dev_ocfr_LT_spring   (altitude, latitude, local_time) float64 ...

# MULTI-YEARS - MAGNETIC Lat Lon - Processing (all seasons)

In [ ]:
####################################      M A G N E T I C  Lat Lon coordinates     ##############################################
####################################     M U L T I - Y E A R S  OF  D A T A    ##############################################
####################################           PROCESSED SEPARATELY            ##############################################

# Define evalue_threshold for each season
evalue_thresholds = {
    'winter': 1.50998e+03,
    'spring': 1.50998e+03,
    'summer': 1.50998e+03,
    'autumn': 2.39315e+03
}

s4max_threshold = 0.35

#Arras and Wickert 2018 s4max > 0.2
#Yue et al 2016 s4max > 0.3
#Yue et al 2015 s4max > 0.3
#Ray 2006 s4max > 0.4
#Ko and Yeh 2010 'strong scintillations' defined as smax >= 0.5

###########################################################################################################################

year_desired_months = [ [3, 4, 5] , [6, 7, 8], [9, 10, 11], [12, 1,2] ]
all_desired_months = []

for d, seasonmonths in enumerate(year_desired_months):

    if set(seasonmonths) <= {3, 4, 5}:
        all_desired_months = [ [3, 4, 5] ]
        years_set = [ '2007' , '2008', '2009', '2010', '2011', '2012', '2013' ]
        MAM_date_sets = [
            ['2007-03-01', '2007-05-31'],
            ['2008-03-01', '2008-05-31'],
            ['2009-03-01', '2009-05-31'],
            ['2010-03-01', '2010-05-31'],
            ['2011-03-01', '2011-05-31'],
            ['2012-03-01', '2012-05-31'],
            ['2013-03-01', '2013-05-31']  
            ]      # x 7 years
        target_date_sets = [[pd.to_datetime(date) for date in set_dates] for set_dates in MAM_date_sets]

    elif set(seasonmonths) <= {6, 7, 8}:
        all_desired_months = [ [6, 7, 8] ]
        years_set = [ '2007', '2008', '2009', '2010', '2011', '2012', '2013'  ]
        JJA_date_sets = [
            ['2007-06-01', '2007-08-31'],
            ['2008-06-01', '2008-08-31'],
            ['2009-06-01', '2009-08-31'],
            ['2010-06-01', '2010-08-31'],
            ['2011-06-01', '2011-08-31'],
            ['2012-06-01', '2012-08-31'],
            ['2013-06-01', '2013-08-31']  
            ]      # x 7 years
        target_date_sets = [[pd.to_datetime(date) for date in set_dates] for set_dates in JJA_date_sets]

    elif set(seasonmonths) <= {9, 10, 11}:
        all_desired_months = [ [9, 10, 11] ]
        years_set = [ '2007', '2008', '2009', '2010', '2011', '2012', '2013' ]
        SON_date_sets = [
            ['2007-09-01', '2007-11-30'],
            ['2008-09-01', '2008-11-30'],
            ['2009-09-01', '2009-11-30'],
            ['2010-09-01', '2010-11-30'],
            ['2011-09-01', '2011-11-30'],
            ['2012-09-01', '2012-11-30'],
            ['2013-09-01', '2013-11-30']  ]      # x 7 years
        target_date_sets = [[pd.to_datetime(date) for date in set_dates] for set_dates in SON_date_sets]

    elif set(seasonmonths) <= {12, 1, 2}:
        all_desired_months = [ [12, 1, 2] ]
        years_set = [ '2007-8', '2008-9', '2009-10', '2010-11', '2011-12', '2012-13' ]
        DJF_date_sets = [
            ['2007-12-01', '2008-02-29'],
            ['2008-12-01', '2009-02-28'],
            ['2009-12-01', '2010-02-28'],
            ['2010-12-01', '2011-02-28'],
            ['2011-12-01', '2012-02-29'],
            ['2012-12-01', '2013-02-28']  ]      # x 6 years
        target_date_sets = [[pd.to_datetime(date) for date in set_dates] for set_dates in DJF_date_sets]
    else:
        print('error error error')

    # Convert 'time' coordinate values to datetime objects
    time_values = pd.to_datetime(ds['time'].values)

    # Find the index of the first value for the first date and the last index for the second date in each set
    indices_for_target_dates_sets = [
        [np.where(time_values >= set_dates[0])[0].min(), np.where(time_values <= set_dates[1])[0].max()] for set_dates in target_date_sets
    ]

    # Display the indices for each set of target dates
    for i, indices_for_target_dates in enumerate(indices_for_target_dates_sets):
        print(f'{years_set[i]} {all_desired_months}:')
        print(f"Indices for Target Dates Set {i + 1}: {indices_for_target_dates}")


        
    for i, indices_for_target_dates in enumerate(indices_for_target_dates_sets):

        dssl_alt = ds['altitude_var'][indices_for_target_dates[0]: indices_for_target_dates[1] + 1]
        dssl_mlat = ds['mlat'][indices_for_target_dates[0]: indices_for_target_dates[1] + 1]
        dssl_mlon = ds['mlon'][indices_for_target_dates[0]: indices_for_target_dates[1] + 1]
        dssl_LT = ds["LT"][indices_for_target_dates[0]: indices_for_target_dates[1] + 1]
        dssl_s4max = ds['s4max'][indices_for_target_dates[0]: indices_for_target_dates[1] + 1]
        dssl_time = ds["time"][indices_for_target_dates[0]: indices_for_target_dates[1] + 1]

        year = years_set[i]
        print('Year ' + str(year))

        # Loop over each set of desired months
        for desired_months in all_desired_months:
            print('Months ' + str(desired_months))
            averaging_period = ""
            if len(desired_months) == 1:
                averaging_period = "Monthly"
            elif len(desired_months) == 2:
                averaging_period = "Two-Month"
            elif len(desired_months) == 3:
                averaging_period = "Three-Month"
            else:
                averaging_period = "Unknown"

            season = ""
            if set(desired_months) <= {12, 1, 2}:
                season = "winter"
            elif set(desired_months) <= {3, 4, 5}:
                season = "spring"
            elif set(desired_months) <= {6, 7, 8}:
                season = "summer"
            elif set(desired_months) <= {9, 10, 11}:
                season = "autumn"
            else:
                season = "unknown"

            # Access the appropriate evalue_threshold for the current season
            current_evalue_threshold = evalue_thresholds[season]
            
            freq_mlat_LT = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))
            nmeas_mlat_LT = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))

            freq_mlat_mlon = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))
            nmeas_mlat_mlon = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))

            nfile = len(dssl_mlat)
            ds_length = np.arange(nfile)

            ###########################################################################################################################
            for i in tqdm(ds_length):
                current_altitude = dssl_alt[i]
                current_mlat = dssl_mlat[i]
                current_mlon = dssl_mlon[i]
                current_LT = dssl_LT[i]
                current_s4max = dssl_s4max[i]
                current_time = dssl_time[i]

                #current_month = int(np.datetime64(current_time, 'M').astype('M8[M]').view('i8')) % 12 + 1
                #current_month = int(current_time.astype('M8[M]').view('i8')) % 12 + 1
                current_month = int(current_time.dt.month)  # Extract month as an integer

                evalue = ( ( ( np.sqrt(13.64*current_s4max) + 1.2 ) / 8.98 )**2 ) * 1e6

                if current_month in desired_months:
                    if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[0]:
                        altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges, right=False) - 1
                        mlat_bin_index = np.digitize(current_mlat, lat_bin_edges) - 1
                        mlon_bin_index = np.digitize(current_mlon, lon_bin_edges) - 1
                        if current_mlon == lon_bin_edges[-1]:
                            mlon_bin_index = len(lon_bin_edges) - 2
                        LT_bin_index = np.digitize(current_LT, LT_bin_edges) - 1
                        if current_LT == LT_bin_edges[-1]:
                            LT_bin_index = len(LT_bin_edges) - 2

                        nmeas_mlat_mlon[altitude_bin_index, mlat_bin_index, mlon_bin_index] += 1.
                        nmeas_mlat_LT[altitude_bin_index, mlat_bin_index, LT_bin_index] += 1.

                        if current_s4max > s4max_threshold and evalue > current_evalue_threshold:
                            freq_mlat_mlon[altitude_bin_index, mlat_bin_index, mlon_bin_index] += 1.
                            freq_mlat_LT[altitude_bin_index, mlat_bin_index, LT_bin_index] += 1.

            ###########################################################################################################################
            # Calculate occurence frequency
            ocfr_mlat_mlon = np.full((Zavg_sl_num_bins, lat_num_bins, lon_num_bins), 0.)
            ocfr_mlat_LT = np.full((Zavg_sl_num_bins, lat_num_bins, LT_num_bins), 0.)

            # Check if arrays are equal to 0
            lon_mask = (freq_mlat_mlon != 0.) & (nmeas_mlat_mlon != 0.)
            LT_mask = (freq_mlat_LT != 0.) & (nmeas_mlat_LT != 0.)

            # Calculate ocfr_lon and ocfr_LT only where the mask is True
            if lon_mask.any():
                ocfr_mlat_mlon[lon_mask] = (freq_mlat_mlon[lon_mask] / nmeas_mlat_mlon[lon_mask]) * 100.0

            if LT_mask.any():
                ocfr_mlat_LT[LT_mask] = (freq_mlat_LT[LT_mask] / nmeas_mlat_LT[LT_mask]) * 100.0

            ###########################################################################################################################
            # Save occurrence frequency to nc file
            Ocfr_ds = xr.Dataset(
                data_vars={
                    "ocfr_mlat_mlon": (['altitude', 'mag_lat', 'mag_lon'], ocfr_mlat_mlon),
                    "ocfr_mlat_LT": (['altitude', 'mag_lat', 'local_time'], ocfr_mlat_LT)
                },
                coords={
                    "altitude": Zavg_sl_bin_midpoints,
                    "mag_lat": lat_bin_midpoints,
                    "mag_lon": lon_bin_midpoints,
                    "local_time": LT_bin_midpoints,
                },
                attrs={
                    'year': year,
                    'averaging_period': averaging_period,
                    'season': season
                }
            )

            output_directory = "./Nc_Files/s4max/e_criteria/individual_yrs/Mag_Coords/"
            output_file = f"{output_directory}Mag_Ocfr_{year}_{season}_s4max_{s4max_threshold}.nc"
            if fivedegbins == 1:
                output_file = f"{output_directory}Mag_Ocfr_{year}_{season}_5d_s4max_{s4max_threshold}.nc"
            Ocfr_ds.to_netcdf(output_file)

            try:
                Ocfr_ds.to_netcdf(output_file)
                print(f"File saved successfully: {output_file}")
            except Exception as e:
                print(f"Error saving the file: {e}")
            finally:
                Ocfr_ds.close()

        ###########################################################################################################################


2007 [[3, 4, 5]]:
Indices for Target Dates Set 1: [256970, 739018]
2008 [[3, 4, 5]]:
Indices for Target Dates Set 2: [1652474, 2017396]
2009 [[3, 4, 5]]:
Indices for Target Dates Set 3: [3309162, 3742893]
2010 [[3, 4, 5]]:
Indices for Target Dates Set 4: [4945053, 5300255]
2011 [[3, 4, 5]]:
Indices for Target Dates Set 5: [6115790, 6415241]
2012 [[3, 4, 5]]:
Indices for Target Dates Set 6: [7203654, 7443213]
2013 [[3, 4, 5]]:
Indices for Target Dates Set 7: [8236216, 8522674]
Year 2007
Months [3, 4, 5]


100%|████████████████████████████████████████████████████████████████████████████████| 482049/482049 [44:33<00:00, 180.32it/s]


File saved successfully: ./Nc_Files/s4max/e_criteria/individual_yrs/Mag_Coords/Mag_Ocfr_2007_spring_5d_s4max_0.35.nc
Year 2008
Months [3, 4, 5]


100%|████████████████████████████████████████████████████████████████████████████████| 364923/364923 [29:11<00:00, 208.41it/s]


File saved successfully: ./Nc_Files/s4max/e_criteria/individual_yrs/Mag_Coords/Mag_Ocfr_2008_spring_5d_s4max_0.35.nc
Year 2009
Months [3, 4, 5]


...

100%|████████████████████████████████████████████████████████████████████████████████| 380860/380860 [30:53<00:00, 205.44it/s]


File saved successfully: ./Nc_Files/s4max/e_criteria/individual_yrs/Mag_Coords/Mag_Ocfr_2007_summer_5d_s4max_0.35.nc
Year 2008
Months [6, 7, 8]


100%|████████████████████████████████████████████████████████████████████████████████| 434071/434071 [37:30<00:00, 192.89it/s]


File saved successfully: ./Nc_Files/s4max/e_criteria/individual_yrs/Mag_Coords/Mag_Ocfr_2008_summer_5d_s4max_0.35.nc
Year 2009
Months [6, 7, 8]


 93%|██████████████████████████████████████████████████████████████████████████▎     | 371388/400139 [30:42<02:22, 201.79it/s]

# MULTI-YEARS - MAGNETIC Lat Lon - Standard Deviation over time 

In [14]:
# Standard Deviation over time (multiple years)

s4max_threshold = 0.35

# Define seasons, years, and output directory
seasons = [
    #'spring'#,
    # 'summer'#,
   #  'autumn'#,
     'winter'
]
years = list(range(2007, 2014))  # For spring, summer, and autumn
years_winter = list(range(2007, 2013))  # For winter
file_directory = "./Nc_Files/s4max/e_criteria/individual_yrs/Mag_Coords/"

# Initialize an empty dataset to store standard deviations
std_dev_dataset = xr.Dataset()

# Loop through seasons
for season in seasons:
    # Define a list to store datasets for each year
    datasets = []

    # Loop through years
    if season == 'winter':
        years_to_loop = years_winter
    else:
        years_to_loop = years

    for year in years_to_loop:
        # Define the file pattern
        if season == 'winter':
            yr_last_two_digits = str(int(str(year)[-2:]) + 1)
            file_pattern = f"{file_directory}Mag_Ocfr_{year}-{yr_last_two_digits}_{season}_5d_s4max_{s4max_threshold}.nc"
        else:
            file_pattern = f"{file_directory}Mag_Ocfr_{year}_{season}_5d_s4max_{s4max_threshold}.nc"

        # Open the dataset
        ocfr_ds = xr.open_dataset(file_pattern)

        # Add a 'time' coordinate representing the year
        ocfr_ds['time'] = xr.DataArray([year], dims='time', coords={'time': [year]})

        # Append the dataset to the list
        datasets.append(ocfr_ds)

    # Concatenate along the time dimension
    combined_ds = xr.concat(datasets, dim='time')

    # Calculate the standard deviation for ocfr_lon and ocfr_LT
    std_dev_mlat_mlon = combined_ds['ocfr_mlat_mlon'].std(dim='time')
    std_dev_mlat_LT = combined_ds['ocfr_mlat_LT'].std(dim='time')

    # Add the standard deviations to the result dataset
    std_dev_dataset[f'std_dev_ocfr_mlat_mlon_{season}'] = std_dev_mlat_mlon
    std_dev_dataset[f'std_dev_ocfr_mlat_LT_{season}'] = std_dev_mlat_LT

# Print or save the result dataset
#print(std_dev_dataset)

output_directory = "./Nc_Files/s4max/e_criteria/individual_yrs/Mag_Coords/"
output_file = f"{output_directory}Mag_Ocfr_stddev_{season}_s4max_{s4max_threshold}.nc"

try:
    std_dev_dataset.to_netcdf(output_file)
    print(f"File saved successfully: {output_file}")
except Exception as e:
    print(f"Error saving the file: {e}")
finally:
    std_dev_dataset.close()
    


File saved successfully: ./Nc_Files/s4max/e_criteria/individual_yrs/Mag_Coords/Mag_Ocfr_stddev_winter_s4max_0.35.nc
